In [1]:
%reload_ext autoreload 
%autoreload 2

import ktwo19.io
ktwo19.io.load_times()

ktwo19/io.py:40: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  times = pd.read_csv(sio(s),sep='\s*',comment='#')


,i_planet,i_epoch,tc,tc_err
0,1,0,1980.38403,0.00044
1,1,1,1988.30421,0.00061
2,1,2,1996.22197,0.00048
3,1,3,2004.13851,0.00066
4,1,4,2012.06161,0.00042
5,1,5,2019.97953,0.00066
6,1,6,2027.90124,0.00068
7,1,7,2035.82024,0.00041
8,1,8,2043.73925,0.00054
9,1,9,2051.65861,0.00059


In [6]:
import lmfit
import pandas as pd
from astropy.time import Time
from astropy import units as u
#import ttv.ttvfastfit
#reload(ttv.ttvfastfit)
%pylab inline
bjd0 = 2454833
from cStringIO import StringIO as sio
#times = pd.read_csv(sio(s),sep='\s*',comment='#')
#times['time'] -= bjd0

run_mcmc=False

Populating the interactive namespace from numpy and matplotlib


In [13]:
# Field 1 observability
spitzer_tstart = Time('2017-03-21',format='iso')
spitzer_tstop = Time('2017-04-28',format='iso')

In [14]:
#target = FixedTarget.from_name('K2-19')
#target.name = 'K2-19b'
#par = planets.ix['K2-19b']
#T0 = Time(par.T0 + bjd0,format='jd')
#T0_arr = future_transits(par.P,T0,tstart=tstart,tstop=tstop)

In [15]:
# Armstrong+15 lithwick exploration
# per1 = 7.921 + 0.028 - 0.003 d
# per2 = 11.91 + 0.12 - 0.12
# tc1 = 813.38345 + 2456000 +/- 0.00036
# tc2 = 817.2759 + 2456000 ± 0.0012

# Nespral
# Pb = 7.91923 ± 0.00020
# Pc = 11.90829+0.00233

# Narita
# Pb 7.920994 ± 0.000071 # Narita
# Pc 12.0028 ± 0.0092 # Nartia, however, they note two minima 

# Nartia second minimum
# Pc 11.7748 ± 0.0142

# Sinokoff/Dai
# 7.91940 ± 0.00005
# 11.90715 ± 0.00150

# Barros+15 photo-dynamical model
# tc1 = 7.921101(69) + Epoch + 2456813.3767(21)
# tc2 = 11.90727(58) × Epoch + 2456817.2755(22)


# C1 ~8 2014, so 2.5 years from observations
# Plan: 

In [16]:
params = ttv.ttvfastfit.Parameters(2,'mu per ecosw esinw inc longnode T')
params.add_many(
    ('per1', 7.91923, True, None, None, None),
    ('per2', 11.90829, True, None, None, None),
    ('T1', 2456813.38403-bjd0,True, None, None, None),
    ('T2', 2456817.27227-bjd0,True, None, None, None),
    ('mu1', 20.5 / (3.33e5*0.918), True, 0, None, None),
    ('mu2', 21.4 / (3.33e5*0.918), True, 0, None, None),
    ('ecosw1', -0.01, True, -0.1,0.1, None),
    ('ecosw2',0.01, True, -0.1,0.1, None),
    ('esinw1',0.01, True, -0.1,0.1, None),
    ('esinw2', 0.00, True, -0.1,0.1, None),
    ('inc1', 90, False, None, None, None),
    ('inc2', 90, False, None, None, None),
    ('longnode1',  180, False, None, None, None),
    ('longnode2',  180, False, None, None, None),
)
t_start = times.time.min() - 1 
t_stop = t_start + 2000 
dt = 0.1
mod = ttv.ttvfastfit.Model(t_start,t_stop,dt)
df = mod(params)
times.index = [times.i_planet,times.i_epoch]

def resid(params):
    times_mod = mod(params)
    times_mod = pd.DataFrame(times_mod)
    times_mod.index = [times_mod.i_planet,times_mod.i_epoch]
    resid = (times_mod.time - times.time).dropna()
    resid_err = times.ix[resid.index].time_err
    nresid = resid / resid_err
    nresid = np.array(nresid)
    return nresid

def logprob(params):
    _logprob = -0.5 * np.sum(resid(params)**2)
    e1 = np.sqrt(params['ecosw1']**2  + params['esinw1']**2 )
    e2 = np.sqrt(params['ecosw2']**2  + params['esinw2']**2 )
    if e1 > 0.05:
        return -np.inf
    if e2 > 0.05:
        return -np.inf

    mu_prior_mean = 25 / 3.33e5
    mu_prior_sigma = 5 / 3.33e5
    _logprob += -0.5 * (params['mu1'] - mu_prior_mean)**2/mu_prior_sigma**2
    _logprob += -0.5 * (params['mu2'] - mu_prior_mean)**2/mu_prior_sigma**2
    return _logprob    
    
    
def neglogprob(params):
    print params.pretty_print()
    return -1.0 * logprob(params)

nwalkers=50
varykeys = [k for k in params.keys() if params[k].vary]
pos = np.array([params[k].value for k in varykeys]) #
pos = np.vstack([pos]*nwalkers) # nwalkers x nvarys
pos[:,varykeys.index('per1')] += np.random.randn(nwalkers) * 1e-5
pos[:,varykeys.index('per2')] += np.random.randn(nwalkers) * 1e-5
pos[:,varykeys.index('mu1')] += np.random.randn(nwalkers) * 1e-5
pos[:,varykeys.index('mu2')] += np.random.randn(nwalkers) * 1e-5
pos[:,varykeys.index('T1')] += np.random.randn(nwalkers) * 1e-5
pos[:,varykeys.index('T2')] += np.random.randn(nwalkers) * 1e-5
pos[:,varykeys.index('ecosw1')] += np.random.randn(nwalkers) * 1e-2
pos[:,varykeys.index('ecosw2')] += np.random.randn(nwalkers) * 1e-2
pos[:,varykeys.index('esinw1')] += np.random.randn(nwalkers) * 1e-2
pos[:,varykeys.index('esinw2')] += np.random.randn(nwalkers) * 1e-2

mi = lmfit.minimize(resid, params, method='Nelder')
mini = lmfit.Minimizer(logprob, mi.params)
#res = mini.emcee(burn=300, steps=600, params=mi.params,nwalkers=nwalkers,pos=pos)

In [17]:
res.flatchain.to_hdf('chain_K2-19.hdf','chain')

NameError: name 'res' is not defined

In [8]:
chain = pd.read_hdf('chain_K2-19.hdf','chain')

In [7]:
import ttv.plotting
reload(ttv.plotting)
ephem_list = [dict(P=7.91940,t0=1913.3837+2454900-bjd0), dict(P=11.90715, t0=1917.2755+2454900-bjd0)]
kwarg_list = [dict(color='r',alpha=0.1),dict(color='b',alpha=0.1)]
times_mod_list = ttv.plotting.ttvfast_sample_chain(params, mod, chain_samp)

ttv.plotting.ttvfast_plot_sample_chain(times_mod_list, ephem_list, kwarg_list)

g = times_mod_list.groupby(['i_planet','i_epoch'],as_index=False)
times_per = g.first()
times_per = times_per.drop(['index','sample'],axis=1)
times_per['time95'] = g.apply(lambda x: np.percentile(x['time'],95)).tolist()
times_per['time05'] = g.apply(lambda x: np.percentile(x['time'],5)).tolist()

i = 0
for i_planet in [1,2]:
    times_per_i = times_per[times_per.i_planet==i_planet]
    ttv.plotting.errorbar_omc(times_per_i.i_epoch,times_per_i.time05, ephem_list[i]['P'], ephem_list[i]['t0'],lw=2,color='k')
    ttv.plotting.errorbar_omc(times_per_i.i_epoch,times_per_i.time95, ephem_list[i]['P'], ephem_list[i]['t0'],lw=2,color='k')
    i+=1
    
ylim( -0.4, 0.4)
xlabel('BJD - 2454833')
ylabel('O-C (days)') 


NameError: name 'params' is not defined

In [108]:
from astropy.time import Time
times_per['time05_iso'] = Time(times_per.time05 + 2454833,format='jd').iso
times_per['time05_jd'] = Time(times_per.time05 + 2454833,format='jd').jd
times_per['time95_iso'] = Time(times_per.time95 + 2454833,format='jd').iso
times_per['time95_jd'] = Time(times_per.time95 + 2454833,format='jd').jd

start_time =  Time(datetime.datetime(2017,4,13)).jd
stop_time = Time(datetime.datetime(2017,5,30)).jd

times_per[(times_per.i_planet==1) & (times_per['time05_jd'].between(start_time,stop_time))]

,i_planet,i_epoch,time,time95,time05,time05_iso,time05_jd,time95_iso,time95_jd
132,1,132,3026.050002,3026.050302,3025.815073,2017-04-15 07:33:42.280,2.457859e+06,2017-04-15 13:12:26.102,2.457859e+06
133,1,133,3033.973119,3033.973167,3033.734256,2017-04-23 05:37:19.729,2.457867e+06,2017-04-23 11:21:21.653,2.457867e+06
134,1,134,3041.902660,3041.903881,3041.660013,2017-05-01 03:50:25.102,2.457875e+06,2017-05-01 09:41:35.338,2.457875e+06
135,1,135,3049.825090,3049.826377,3049.580398,2017-05-09 01:55:46.358,2.457883e+06,2017-05-09 07:49:58.952,2.457883e+06
136,1,136,3057.748166,3057.749538,3057.500264,2017-05-17 00:00:22.801,2.457891e+06,2017-05-17 05:59:20.068,2.457891e+06
137,1,137,3065.677748,3065.680053,3065.427954,2017-05-24 22:16:15.231,2.457898e+06,2017-05-25 04:19:16.591,2.457899e+06


In [33]:
from astropy.time import Time
Time(2457865.170708,format='jd').iso

'2017-04-21 16:05:49.171'

In [103]:
# Based on the best available data, K2-19b the mid tranist time is between 

# 2017-04-15: 07:33--13:12 -> T1 = 06:03--11:42, T4=09:03--14:42
# 2017-04-23: 05:37--11:21 -> T1 = 04:07--09:51, T4=07:07--12:51
# 2017-05-01: 03:50--09:41 -> T1 = 02:20--08:11, T4=05:20--11:11

2457866.5